In [1]:
pip install underthesea pandas regex --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from underthesea import text_normalize, word_tokenize

In [3]:
df_train = pd.read_csv( "./df_train.csv", encoding="utf-8")
df_dev   = pd.read_csv( "./df_dev.csv",   encoding="utf-8")
df_test  = pd.read_csv( "./df_test.csv",  encoding="utf-8")

print("Train columns:", df_train.columns.tolist())
print("Dev   columns:", df_dev.columns.tolist())
print("Test  columns:", df_test.columns.tolist())

Train columns: ['content', 'sentiment', 'topic']
Dev   columns: ['content', 'sentiment', 'topic']
Test  columns: ['content', 'sentiment', 'topic']


In [4]:
for df in (df_train, df_dev, df_test):
    if 'sentiment' in df.columns:
        df.rename(columns={'sentiment':'label'}, inplace=True)
    assert 'content' in df.columns and 'label' in df.columns, \
        f"Thiếu cột ở frame với columns={df.columns.tolist()}"
    df['content'] = df['content'].fillna('').astype(str)

In [5]:
print("Loaded splits:", len(df_train), len(df_dev), len(df_test))

Loaded splits: 11426 1583 3166


In [6]:
df_misspell = pd.read_csv('./vietnamese-misspell.csv')
misspell_dict = dict(zip(df_misspell['wrong'], df_misspell['right']))

In [7]:
teencode_map = {}
with open("./vietnamese-teencode.txt", "r", encoding="utf8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            teencode_map[parts[0]] = parts[1]
print("Ví dụ teencode_map:", list(teencode_map.items())[:10])

Ví dụ teencode_map: [('ctrai', 'con'), ('khôg', 'không'), ('bme', 'bố'), ('cta', 'chúng'), ('mih', 'mình'), ('mqh', 'mối'), ('cgai', 'con'), ('nhữg', 'những'), ('mng', 'mọi'), ('svtn', 'sinh')]


In [8]:
emoticon_dict = {
    ":)": "☺️",
    ":))": "☺️",
    ":)))": "☺️",
    "=))": "😄",
    "=)": "😄",
    ":D": "😀",
    ":DD": "😁",
    ":<": "😞",
    ":(": "☹️",
    ":'(": "😢",
    ":'(": "😢",
    ":')": "😂",
    ":')": "😂",
    "<3": "❤️",
    "</3": "💔",
    ":P": "😛",
    ":p": "😛",
    ":O": "😲",
    ":o": "😲",
    ";)": "😉",
    ";-)": "😉",
    ":3": "😺",
    ":^)": "😊",
    "^_^": "😊",
    "-_-": "😑",
    ">_<": "😣",
    "XD": "😆",
    "xD": "😆",
    "T_T": "😭",
    ";_;": "😭",
    ":|": "😐",
    ":/": "😕",
    ":-/": "😕",
    ":-\\": "😕",
    ":'D": "😆",
    ":'D": "😆",
    ":-*": "😘",
    ":*": "😘",
    "<<": "😓",
    ">_>": "😒",
    "<_<": "😒",
    "^\\^": "😆",
    "\\^_^/": "🎉",
    "*^_^*": "🎉",
    "\\o/": "🙌",
    "\\O/": "🙌",
    "O_O": "😳",
    "o_o": "😳",
    ">:O": "😠",
    "^^": "☺️"
}


In [9]:
def standardize_emoticon(text):
    # Gộp các chuỗi emoticon thường gặp về chuẩn (gộp lặp, ví dụ ^^ ^^ ^^ -> ^^)
    # Gộp emoticon kiểu :) về 1
    text = re.sub(r'((:\)+))', ':)', text)
    text = re.sub(r'((=\)+))', '=)', text)
    text = re.sub(r'((\^_?\^)+)', '^^', text)        # ^^, ^_^, ^^ ^^, ...
    text = re.sub(r'(<3+)', '<3', text)
    text = re.sub(r'(\)+)', ')', text)
    text = re.sub(r'(\(+)', '(', text)
    # Gộp mọi chuỗi ^^ liên tiếp về 1 ^^ (kể cả có cách ra)
    text = re.sub(r'(\^\^)(\s+\^\^)+', '^^', text)
    # Gộp ((
    text = re.sub(r'(\(+)', '(', text)
    text = re.sub(r'(\)+)', ')', text)
    return text

In [10]:
def standardize_emoticon(text):
    # Gộp các chuỗi emoticon thường gặp về chuẩn (gộp lặp, ví dụ ^^ ^^ ^^ -> ^^)
    # Gộp emoticon kiểu :) về 1
    text = re.sub(r'((:\)+))', ':)', text)
    text = re.sub(r'((=\)+))', '=)', text)
    text = re.sub(r'((\^_?\^)+)', '^^', text)        # ^^, ^_^, ^^ ^^, ...
    text = re.sub(r'(<3+)', '<3', text)
    text = re.sub(r'(\)+)', ')', text)
    text = re.sub(r'(\(+)', '(', text)
    # Gộp mọi chuỗi ^^ liên tiếp về 1 ^^ (kể cả có cách ra)
    text = re.sub(r'(\^\^)(\s+\^\^)+', '^^', text)
    # Gộp ((
    text = re.sub(r'(\(+)', '(', text)
    text = re.sub(r'(\)+)', ')', text)
    return text

In [11]:
def convert_emoticon(text, emoticon_dict):
    # Duyệt emoticon dài trước
    for emo in sorted(emoticon_dict, key=len, reverse=True):
        # Chỉ thay thế khi là nguyên một cụm (dùng word boundary nếu cần)
        text = re.sub(re.escape(emo) + r'(?=\s|$)', emoticon_dict[emo], text)
    return text

In [12]:
def standardize_word(text, misspell_dict):
    # Gộp ký tự kéo dài (đẹpppp -> đẹp)
    text = re.sub(r'(\w)\1{2,}', r'\1', text)
    # Chuẩn hóa chính tả/viết tắt
    for wrong, right in misspell_dict.items():
        text = re.sub(r'\b' + re.escape(wrong) + r'\b', right, text)
    return text

In [13]:
teencode_map = {}
with open("./vietnamese-teencode.txt", "r", encoding="utf8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            teencode_map[parts[0]] = parts[1]

In [14]:
import re

def remove_duplicate_emoji(text):
    # Xóa emoji trùng lặp liên tiếp (vd: 😄😄😄 -> 😄)
    emoji_pattern = re.compile(
        r'([\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF])\1+'
    )
    return emoji_pattern.sub(r'\1', text)


In [15]:
def preprocess_text(text, emoticon_dict, misspell_dict):
    text = str(text)
    text = standardize_emoticon(text)
    text = convert_emoticon(text, emoticon_dict)
    text = remove_duplicate_emoji(text)
    for abbr, full in teencode_map.items():
        pattern = rf"\b{re.escape(abbr)}\b"
        text = re.sub(pattern, full, text, flags=re.IGNORECASE)
    text = standardize_word(text, misspell_dict)
    return text

In [16]:
text = "ước gì sau này về già vẫn có thể như cụ này :)) :)))) :)))"
text = preprocess_text(text, emoticon_dict, misspell_dict)
print(text) 

ước gì sau này về già vẫn có thể như cụ này ☺️ ☺️ ☺️


In [17]:
df_test['content_clean'] = df_test['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_test[['content', 'content_clean']].head(10)

,content,content_clean
0,nói tiếng anh lưu loát .,nói tiếng anh lưu loát .
1,giáo viên rất vui tính .,giáo viên rất vui tính .
2,cô max có tâm .,cô max có tâm .
3,"giảng bài thu hút , dí dỏm .","giảng bài thu hút , dí dỏm ."
4,"giáo viên không giảng dạy kiến thức , hướng dẫ...","giáo viên không giảng dạy kiến thức , hướng dẫ..."
5,thầy dạy nhiệt tình và tâm huyết .,thầy dạy nhiệt tình và tâm huyết .
6,tính điểm thi đua các nhóm .,tính điểm thi đua các nhóm .
7,thầy nhiệt tình giảng lại cho học sinh .,thầy nhiệt tình giảng lại cho học sinh .
8,có đôi lúc nói hơi nhanh làm sinh viên không t...,có đôi lúc nói hơi nhanh làm sinh viên không t...
9,"giảng dạy nhiệt tình , liên hệ thực tế khá nhi...","giảng dạy nhiệt tình , liên hệ thực tế khá nhi..."


In [18]:
df_train['content_clean'] = df_train['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_train[['content', 'content_clean']].head(10)

,content,content_clean
0,slide giáo trình đầy đủ .,slide giáo trình đầy đủ .
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .","nhiệt tình giảng dạy , gần gũi với sinh viên ."
2,đi học đầy đủ full điểm chuyên cần .,đi học đầy đủ full điểm chuyên cần .
3,chưa áp dụng công nghệ thông tin và các thiết ...,chưa áp dụng công nghệ thông tin và các thiết ...
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...","thầy giảng bài hay , có nhiều bài tập ví dụ ng..."
5,"giảng viên đảm bảo thời gian lên lớp , tích cự...","giảng viên đảm bảo thời gian lên lớp , tích cự..."
6,"em sẽ nợ môn này , nhưng em sẽ học lại ở các h...","em sẽ nợ môn này , nhưng em sẽ học lại ở các h..."
7,"thời lượng học quá dài , không đảm bảo tiếp th...","thời lượng học quá dài , không đảm bảo tiếp th..."
8,"nội dung môn học có phần thiếu trọng tâm , hầu...","nội dung môn học có phần thiếu trọng tâm , hầu..."
9,cần nói rõ hơn bằng cách trình bày lên bảng th...,cần nói rõ hơn bằng cách trình bày lên bảng th...


In [19]:
df_dev['content_clean'] = df_dev['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_dev[['content', 'content_clean']].head(10)

,content,content_clean
0,giáo trình chưa cụ thể .,giáo trình chưa cụ thể .
1,giảng buồn ngủ .,giảng buồn ngủ .
2,"giáo viên vui tính , tận tâm .","giáo viên vui tính , tận tâm ."
3,"giảng viên nên giao bài tập nhiều hơn , chia n...","giảng viên nên giao bài tập nhiều hơn , chia n..."
4,"giảng viên cần giảng bài chi tiết hơn , đi sâu...","giảng viên cần giảng bài chi tiết hơn , đi sâu..."
5,nên có giảng viên nước ngoài dạy để sinh viên ...,nên có giảng viên nước ngoài dạy để sinh viên ...
6,nên có bài tập lớn đồ án môn học .,nên có bài tập lớn đồ án môn học .
7,"giảng viên đảm bảo nội dung học , phân tích gi...","giảng viên đảm bảo nội dung học , phân tích gi..."
8,"nêu rõ mục tiêu , mục đích môn học để sinh viê...","nêu rõ mục tiêu , mục đích môn học để sinh viê..."
9,có một số vấn đề nói chưa rõ .,có một số vấn đề nói chưa rõ .


In [20]:
df_train.to_csv("./df_train_clean.csv", index=False)
df_dev  .to_csv("./df_dev_clean.csv",   index=False)
df_test .to_csv("./df_test_clean.csv",  index=False)
print("Cleaned files written to /content/")

Cleaned files written to /content/
